In [1]:
from ipynb.fs.full.Infer_propagation_path_functions import *

from pymongo import MongoClient

In [2]:
client = MongoClient('mongodb://localhost:27017/')

db = client.get_database('tweets')

In [3]:
COLLECTION_NAME = 'data_false_3'
tweets = db[COLLECTION_NAME].find({})
followers = db['followers'].find({})

tweet_user_ids = []
for tweet in tweets:
    tweet_user_ids.append(tweet['user']['id'])

tweet_user_ids

[1379810428197482499,
 1582038943,
 1379810428197482499,
 87612386,
 1379810428197482499,
 1097352785794785280,
 1301024185767125000,
 1195887405494079489,
 1315095273434210305,
 1379810428197482499,
 1379810428197482499,
 1379810428197482499,
 37943674,
 1244994281426878464,
 1041626063577669632,
 1212270170615341056,
 1179943673624190977,
 1379810428197482499,
 1379810428197482499,
 1379810428197482499,
 1174000620275163138,
 114251756,
 1509690282,
 1246852606686830593,
 122298741,
 449750063,
 1379810428197482499,
 1379810428197482499,
 1379810428197482499,
 1379810428197482499,
 4297338075,
 4297338075,
 805079630000443392,
 1379810428197482499,
 1379810428197482499,
 790019230389248000,
 2627013992,
 711844276569681920,
 1379810428197482499,
 936373163272712193,
 103550320,
 1379810428197482499,
 1255016368769916928,
 1302841354461081601,
 1299809641879269377,
 29246937,
 2379472700]

In [9]:
'''
has a root user 1
root user has followers 1
If a root user followers intersects with tweets, they are cascade depth one users 1
if a cascade depth one user followers intersects with tweets, they are cascade depth two users
...
if a cascade depth n-1 user followers intersects with tweets, they are cascade depth n users

Lets say parent user publish a tweet and the child re-tweet that exact tweet.
child timestamp must be after the parent's time stamp
If child has many parents, parent with the most recent time stamp will be the owner of the child tweet  
'''
tweets = db[COLLECTION_NAME].find({})
#get root tweet
root_tweet = set()
for tweet in tweets:
    try:
        root_tweet.add(tweet['retweeted_status']['user']['id'])
    except KeyError as e:
        continue
print(root_tweet)

tweets = db[COLLECTION_NAME].find({})

# Create root node user_id, created_at

if len(root_tweet) == 1:
    for tweet in tweets:
        try:
            user_id = tweet['retweeted_status']['user']['id']
            created_at = tweet['retweeted_status']['created_at']
        except KeyError as e:
            continue
            
    print(user_id, created_at)

    root = Node(name = user_id, created_at = created_at, difference_seconds =  0)
    print(root.name)

    follower = db['followers'].find_one({'user_id' : root.name})
    try:
        print(follower['user_id'])
    except:
        print('None, DB does not have the user: ', root.name)
        
    #get depth 1 users

    if follower['user_id'] == root.name:
        root_followers = follower['followers']           

    root_followers

    tweets = db[COLLECTION_NAME].find({})
    followers = db['followers'].find({})

    tweet_user_ids = []
    for tweet in tweets:
        tweet_user_ids.append(tweet['user']['id'])

    tweet_user_ids

    cascade_depth_one_user_ids = list(set(root_followers) & set(tweet_user_ids))

    len(cascade_depth_one_user_ids)

    tweets = db[COLLECTION_NAME].find({})
    followers = db['followers'].find({})

    # add depth 1 users to the tree
    for user_id in cascade_depth_one_user_ids:
        tweets = db[COLLECTION_NAME].find({})
        for tweet in tweets:
            if tweet['user']['id'] == user_id:
                print(tweet['created_at'])
                difference = difference_datetime(tweet['created_at'], root.created_at).total_seconds()
                print(difference)
                globals()[f'user_{user_id}'] = Node(name = user_id, created_at = tweet['created_at'], difference_seconds = difference , parent = root)


    for pre, fill, node in RenderTree(root):
        print("%s user_id: %s, created_at: %s, difference_seconds: %s" % (pre, node.name, node.created_at, node.difference_seconds))
else:
    multiple_root_users(root_tweet)
#     raise('Cascade has more than one root user')

{4297338075, 201078471, 1244994281426878464, 1509690282, 1174000620275163138, 1212270170615341056, 1255016368769916928, 790019230389248000, 1301024185767125000}


NameError: name 'db' is not defined

In [4]:
users = [4297338075, 201078471, 1244994281426878464, 1509690282, 1174000620275163138, 1212270170615341056, 1255016368769916928, 790019230389248000, 1301024185767125000]

In [5]:
for user in users:
    print(user, get_followers_len(user, db))

4297338075 4580
201078471 1172
1244994281426878464 0
1509690282 2401
1174000620275163138 502
1212270170615341056 1822
1255016368769916928 1076
790019230389248000 75177
1301024185767125000 4015


In [8]:
for user in users:
    print(user, get_cascade_size(user, 'data_false_3', db))

4297338075 2
201078471 1
1244994281426878464 1
1509690282 2
1174000620275163138 1
1212270170615341056 1
1255016368769916928 2
790019230389248000 1
1301024185767125000 1


In [9]:
'''
Check if the text is same
If text is same compare time stamps to infer the propagation path. We assign parent as the earliest tweet and child as parent's followers

'''

"\nCheck if the text is same\nIf text is same compare time stamps to infer the propagation path. We assign parent as the earliest tweet and child as parent's followers\n\n"

In [13]:
# Check if the text is same
db['data_false_3'].distinct('text')

["@BitcoinSampo It's like getting pregnant with twins means you're pregnant for 18 months instead of 9. ;)",
 '@Natalie_KateM Congratulations. Hope you are feeling well. Is it just one? My daughter is 22 months and I’m 18 week… https://t.co/Oggx6qPCs3',
 '@catturd2 AOC also said that if she has any children, she hopes she does not have twins, because she does not want… https://t.co/9qZI3mvSP6',
 '@ramen_chmait My twins are  recipients of this treatment.  At 18 weeks pregnant and born at 31 weeks now 5 and half months old 💙💙',
 'AOC said she hopes she never has twins. She couldn’t imagine being pregnant for 18 months!',
 'Alexandria Ocasio-Cortez has time and again been subject of controversy because of her outspoken nature. https://t.co/qGzlJNdCDr',
 'Did AOC Say She Doesn’t Want to Have Twins as She Can’t Afford to be Pregnant for 18\xa0Months? https://t.co/HtS6EDemw3',
 'Does anyone remember when @ShelbyCairney1 thought you were pregnant for 18 months with twins and got death threats

In [14]:
# Check if the text is same
db['data_false_3'].distinct('retweet_count')

[0, 1, 2, 3]

In [15]:
db['data_false_3'].distinct('retweeted_status.retweet_count')

[1, 2, 3, 4]

In [16]:
db['data_false_3'].distinct('retweeted_status.quoted_status.retweet_count')

[]